Parameter searching: https://www.projectpro.io/recipes/find-optimal-parameters-using-gridsearchcv \
Renaming the last column: https://stackoverflow.com/questions/56479835/rename-only-the-last-column-in-pandas-dataframe-accounting-for-duplicate-header

Preperation

In [1]:
import pandas
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn import preprocessing
from sklearn.svm import SVC

c:\Users\victo\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
#loading in dataset normal, STM and BerTopic
df = pandas.read_csv("G:\Master\Block 3\Thesis\Rogue lite\\rogue_lite_final.csv")
df_STM = pandas.read_csv("G:\Master\Block 3\Thesis\Rogue lite\\features_rogue_lite_STM.csv")
df_Bert = pandas.read_csv("G:\Master\Block 3\Thesis\Rogue lite\\features_RL_Bert_reduction.csv")
df_STM_selected = pandas.read_csv("G:\\Master\\Block 3\\Thesis\\Rogue lite\\RL_featured_selected_STM.csv")
df_Bert_selected = pandas.read_csv("G:\\Master\\Block 3\\Thesis\\Rogue lite\\RL_featured_selected_Bert.csv")

In [3]:
df_STM.columns = [*df_STM.columns[:-1], 'sentiment']

In [4]:
df_STM

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V132,V133,V134,V135,V136,V137,V138,V139,V140,sentiment
0,0.007000,0.000534,0.000154,0.002667,0.000652,0.000063,0.002113,0.000080,0.000242,0.000081,...,0.000209,0.000495,0.000071,0.000106,0.007973,0.000108,0.000894,0.000280,0.000501,0
1,0.010644,0.002984,0.000329,0.005637,0.001644,0.000152,0.002915,0.000395,0.000397,0.000422,...,0.000661,0.003246,0.000197,0.000400,0.025340,0.000795,0.002710,0.002697,0.001717,0
2,0.015769,0.002893,0.000479,0.006606,0.002713,0.000180,0.003004,0.000808,0.000284,0.000342,...,0.000348,0.001685,0.000322,0.000742,0.009953,0.000454,0.002182,0.003498,0.001445,0
3,0.015655,0.002680,0.000505,0.004093,0.001991,0.000202,0.003575,0.000999,0.000404,0.000613,...,0.000525,0.000830,0.000548,0.000810,0.017556,0.000948,0.002389,0.001240,0.001797,0
4,0.010775,0.001805,0.000253,0.001813,0.000964,0.000127,0.001374,0.000900,0.000169,0.000936,...,0.000536,0.001483,0.000685,0.000512,0.009651,0.000751,0.001871,0.000290,0.001706,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24393,0.013314,0.001260,0.000875,0.016534,0.001928,0.000262,0.009763,0.001318,0.001011,0.001031,...,0.000723,0.000392,0.034757,0.000498,0.008095,0.001314,0.003711,0.000701,0.002690,0
24394,0.011417,0.002197,0.000461,0.002321,0.000832,0.000099,0.005855,0.000412,0.000285,0.000660,...,0.001373,0.002943,0.000750,0.000227,0.004817,0.000677,0.001135,0.001971,0.001646,0
24395,0.015171,0.002369,0.001274,0.006574,0.019785,0.000477,0.015730,0.001467,0.004890,0.000823,...,0.001075,0.001369,0.000582,0.000668,0.008259,0.001191,0.002114,0.001990,0.002499,0
24396,0.005745,0.001693,0.001037,0.002862,0.002556,0.000261,0.002940,0.003071,0.000351,0.001119,...,0.000813,0.001951,0.001178,0.000453,0.004981,0.001410,0.001626,0.002465,0.002457,0


In [5]:
df_Bert

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,131,132,133,134,135,136,137,138,139,sentiment
0,0,7.161076e-02,8.032540e-03,3.878027e-02,5.338093e-02,9.668135e-03,1.103647e-01,9.359265e-03,4.005914e-03,2.205113e-03,...,2.481152e-03,6.428300e-04,2.199288e-03,6.305197e-04,2.767314e-03,1.181205e-03,1.606444e-03,5.088521e-03,2.379915e-03,0
1,1,2.493316e-305,9.494455e-307,2.652067e-306,1.078227e-305,5.150457e-307,2.696512e-306,1.653094e-306,7.652599e-307,2.836072e-307,...,7.218034e-307,6.955792e-308,4.692486e-307,6.442587e-308,9.064081e-307,1.632524e-307,2.493031e-307,1.166737e-306,6.489213e-307,0
2,2,9.049841e-02,1.086049e-02,4.392182e-02,7.037843e-02,1.118475e-02,1.439517e-01,1.241098e-02,5.032778e-03,2.956815e-03,...,3.129165e-03,8.568429e-04,2.803146e-03,8.426022e-04,3.448951e-03,1.566979e-03,2.173763e-03,6.936084e-03,3.002028e-03,0
3,3,5.427132e-02,2.668340e-03,8.253728e-03,3.470797e-02,1.473610e-03,1.263733e-02,4.589480e-03,1.888938e-03,7.714401e-04,...,1.670334e-03,1.870692e-04,1.296126e-03,1.756435e-04,1.876603e-03,4.163106e-04,6.744426e-04,8.706816e-03,1.536956e-03,0
4,4,1.557301e-305,1.623180e-306,1.016503e-306,6.989701e-306,2.686663e-307,8.231256e-307,2.393766e-306,5.173294e-307,5.666313e-307,...,6.207140e-307,9.166659e-308,2.013440e-306,7.912663e-308,3.934850e-307,2.630371e-307,1.948564e-306,2.720192e-307,7.043960e-307,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24414,24414,5.892645e-02,3.810099e-01,7.971860e-03,4.219291e-02,2.325838e-03,6.281696e-03,1.334045e-02,3.140196e-03,1.315501e-02,...,2.226765e-03,1.638025e-03,3.035198e-03,1.409934e-03,1.879297e-03,2.122318e-03,3.961188e-03,1.872837e-03,2.317176e-03,0
24415,24415,1.332325e-305,1.174029e-306,1.536448e-306,6.278282e-305,3.829572e-307,2.006488e-306,2.036811e-306,4.497688e-307,3.066523e-307,...,4.350305e-307,7.493255e-308,4.584003e-307,7.223216e-308,3.865857e-307,1.467236e-307,2.824223e-307,8.797701e-307,4.237971e-307,0
24416,24416,7.422404e-305,1.256117e-306,1.412481e-306,1.247427e-305,3.420711e-307,1.285612e-306,4.951853e-306,6.800412e-307,3.894692e-307,...,1.348433e-306,8.032551e-308,1.327652e-306,7.235960e-308,7.230003e-307,2.089216e-307,4.765983e-307,4.826088e-307,1.413702e-306,0
24417,24417,1.963084e-01,4.939740e-03,9.086108e-03,5.982562e-02,2.009785e-03,9.245230e-03,1.011308e-02,3.464405e-03,1.482295e-03,...,4.694588e-03,3.437601e-04,2.754826e-03,3.160989e-04,4.247605e-03,8.322930e-04,1.429033e-03,4.186135e-03,4.152724e-03,0


In [3]:
df_STM_selected

,V1,V2,V3,V4,V5,V7,V8,V9,V10,V11,...,V130,V131,V132,V133,V135,V136,V137,V138,V139,sentiment
0,0.007000,0.000534,0.000154,0.002667,0.000652,0.002113,0.000080,0.000242,0.000081,0.001138,...,0.000860,0.000205,0.000209,0.000495,0.000106,0.007973,0.000108,0.000894,0.000280,0
1,0.010644,0.002984,0.000329,0.005637,0.001644,0.002915,0.000395,0.000397,0.000422,0.002362,...,0.001962,0.002910,0.000661,0.003246,0.000400,0.025340,0.000795,0.002710,0.002697,0
2,0.015769,0.002893,0.000479,0.006606,0.002713,0.003004,0.000808,0.000284,0.000342,0.002354,...,0.002128,0.007629,0.000348,0.001685,0.000742,0.009953,0.000454,0.002182,0.003498,0
3,0.015655,0.002680,0.000505,0.004093,0.001991,0.003575,0.000999,0.000404,0.000613,0.002535,...,0.003274,0.002306,0.000525,0.000830,0.000810,0.017556,0.000948,0.002389,0.001240,0
4,0.010775,0.001805,0.000253,0.001813,0.000964,0.001374,0.000900,0.000169,0.000936,0.001591,...,0.002184,0.001222,0.000536,0.001483,0.000512,0.009651,0.000751,0.001871,0.000290,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24393,0.013314,0.001260,0.000875,0.016534,0.001928,0.009763,0.001318,0.001011,0.001031,0.003830,...,0.004682,0.001442,0.000723,0.000392,0.000498,0.008095,0.001314,0.003711,0.000701,0
24394,0.011417,0.002197,0.000461,0.002321,0.000832,0.005855,0.000412,0.000285,0.000660,0.001376,...,0.001693,0.000262,0.001373,0.002943,0.000227,0.004817,0.000677,0.001135,0.001971,0
24395,0.015171,0.002369,0.001274,0.006574,0.019785,0.015730,0.001467,0.004890,0.000823,0.004310,...,0.047210,0.000744,0.001075,0.001369,0.000668,0.008259,0.001191,0.002114,0.001990,0
24396,0.005745,0.001693,0.001037,0.002862,0.002556,0.002940,0.003071,0.000351,0.001119,0.004964,...,0.011068,0.003066,0.000813,0.001951,0.000453,0.004981,0.001410,0.001626,0.002465,0


In [4]:
df_Bert_selected

,0,5,6,7,8,10,11,12,13,14,...,129,131,132,133,135,136,137,138,139,sentiment
0,7.161076e-02,1.103647e-01,9.359265e-03,4.005914e-03,2.205113e-03,5.262285e-03,1.096440e-02,7.333114e-04,6.724123e-03,2.142866e-03,...,4.367887e-03,2.481152e-03,6.428300e-04,2.199288e-03,2.767314e-03,1.181205e-03,1.606444e-03,5.088521e-03,2.379915e-03,0
1,2.493316e-305,2.696512e-306,1.653094e-306,7.652599e-307,2.836072e-307,1.864821e-306,2.621521e-306,8.483101e-308,1.250345e-306,2.669230e-307,...,2.370830e-307,7.218034e-307,6.955792e-308,4.692486e-307,9.064081e-307,1.632524e-307,2.493031e-307,1.166737e-306,6.489213e-307,0
2,9.049841e-02,1.439517e-01,1.241098e-02,5.032778e-03,2.956815e-03,6.633349e-03,1.388343e-02,9.657169e-04,9.020710e-03,2.878185e-03,...,6.421760e-03,3.129165e-03,8.568429e-04,2.803146e-03,3.448951e-03,1.566979e-03,2.173763e-03,6.936084e-03,3.002028e-03,0
3,5.427132e-02,1.263733e-02,4.589480e-03,1.888938e-03,7.714401e-04,3.404314e-03,6.582414e-03,2.204415e-04,5.009688e-03,7.332205e-04,...,7.307532e-04,1.670334e-03,1.870692e-04,1.296126e-03,1.876603e-03,4.163106e-04,6.744426e-04,8.706816e-03,1.536956e-03,0
4,1.557301e-305,8.231256e-307,2.393766e-306,5.173294e-307,5.666313e-307,4.732682e-307,5.983957e-306,1.032788e-307,4.962158e-307,5.006097e-307,...,1.543384e-307,6.207140e-307,9.166659e-308,2.013440e-306,3.934850e-307,2.630371e-307,1.948564e-306,2.720192e-307,7.043960e-307,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24414,5.892645e-02,6.281696e-03,1.334045e-02,3.140196e-03,1.315501e-02,3.878494e-03,1.322405e-02,1.160679e-03,3.701536e-03,1.780783e-02,...,1.491022e-03,2.226765e-03,1.638025e-03,3.035198e-03,1.879297e-03,2.122318e-03,3.961188e-03,1.872837e-03,2.317176e-03,0
24415,1.332325e-305,2.006488e-306,2.036811e-306,4.497688e-307,3.066523e-307,5.786479e-307,2.100246e-306,7.869169e-308,2.085316e-306,2.990455e-307,...,3.111081e-307,4.350305e-307,7.493255e-308,4.584003e-307,3.865857e-307,1.467236e-307,2.824223e-307,8.797701e-307,4.237971e-307,0
24416,7.422404e-305,1.285612e-306,4.951853e-306,6.800412e-307,3.894692e-307,7.014142e-307,7.436520e-306,9.376090e-308,7.951003e-307,3.597786e-307,...,1.931503e-307,1.348433e-306,8.032551e-308,1.327652e-306,7.230003e-307,2.089216e-307,4.765983e-307,4.826088e-307,1.413702e-306,0
24417,1.963084e-01,9.245230e-03,1.011308e-02,3.464405e-03,1.482295e-03,5.228554e-03,1.715577e-02,4.106989e-04,5.591445e-03,1.389315e-03,...,1.054006e-03,4.694588e-03,3.437601e-04,2.754826e-03,4.247605e-03,8.322930e-04,1.429033e-03,4.186135e-03,4.152724e-03,0


In [4]:
#vectorizing the bag of words
vectorizer_model = CountVectorizer(min_df = 5)


In [5]:
#splitting the columns in X and y
bow_X = vectorizer_model.fit_transform(df["review_text"].values)
bow_y = df["review_score"].values
STM_X = df_STM.drop("sentiment", axis=1)
STM_X = STM_X.values
STM_y = df_STM["sentiment"].values
STM_X_selected = df_STM_selected.drop("sentiment", axis = 1)
STM_y_selected = df_STM_selected["sentiment"].values
Bert_X = df_Bert.drop(["sentiment", "Unnamed: 0"], axis=1)
Bert_X = Bert_X.values
Bert_y = df_Bert["sentiment"].values
Bert_X_selected = df_Bert_selected.drop("sentiment", axis = 1)
Bert_y_selected = df_Bert_selected["sentiment"].values

In [8]:
bow_X

<24403x13703 sparse matrix of type '<class 'numpy.int64'>'
	with 1871985 stored elements in Compressed Sparse Row format>

In [6]:
#splitting the sets
bow_X_train, bow_X_test, bow_y_train, bow_y_test = train_test_split(bow_X, bow_y, random_state = 101)
STM_X_train, STM_X_test, STM_y_train, STM_y_test = train_test_split(STM_X, STM_y, random_state = 101)
Bert_X_train, Bert_X_test, Bert_y_train, Bert_y_test = train_test_split(Bert_X, Bert_y, random_state = 101)
STM_X_selected_train, STM_X_selected_test, STM_y_selected_train, STM_y_selected_test = train_test_split(STM_X_selected,STM_y_selected, random_state=101)
Bert_X_selected_train, Bert_X_selected_test, Bert_y_selected_train, Bert_y_selected_test = train_test_split(Bert_X_selected,Bert_y_selected, random_state=101)

In [7]:
sc_bow = StandardScaler(with_mean=False)
bow_X_train = sc_bow.fit_transform(bow_X_train)
bow_X_test = sc_bow.transform(bow_X_test)

In [8]:
sc_stm = StandardScaler(with_mean=False)
STM_X_train = sc_stm.fit_transform(STM_X_train)
STM_X_test = sc_stm.transform(STM_X_test)

In [9]:
sc_Bert = StandardScaler(with_mean=False)
Bert_X_train = sc_Bert.fit_transform(Bert_X_train)
Bert_X_test = sc_Bert.transform(Bert_X_test)

In [10]:
sc_stm_selected = StandardScaler(with_mean=False)
STM_X_selected_train = sc_stm.fit_transform(STM_X_selected_train)
STM_X_selected_test = sc_stm.transform(STM_X_selected_test)

In [11]:
sc_Bert_selected = StandardScaler(with_mean=False)
Bert_X_selected_train = sc_Bert.fit_transform(Bert_X_selected_train)
Bert_X_selected_test = sc_Bert.transform(Bert_X_selected_test)

In [7]:
le_bow = preprocessing.LabelEncoder()
bow_y_train = le_bow.fit_transform(bow_y_train)
bow_y_test = le_bow.transform(bow_y_test)

In [8]:
le_stm = preprocessing.LabelEncoder()
STM_y_train = le_stm.fit_transform(STM_y_train)
STM_y_test = le_stm.transform(STM_y_test)

In [9]:
le_bert = preprocessing.LabelEncoder()
Bert_y_train = le_bert.fit_transform(Bert_y_train)
Bert_y_test = le_bert.transform(Bert_y_test)

In [10]:
le_stm_selected = preprocessing.LabelEncoder()
STM_y_selected_train = le_stm.fit_transform(STM_y_selected_train)
STM_y_selected_test = le_stm.transform(STM_y_selected_test)

In [11]:
le_bert_selected = preprocessing.LabelEncoder()
Bert_y_selected_train = le_bert.fit_transform(Bert_y_selected_train)
Bert_y_selected_test = le_bert.transform(Bert_y_selected_test)

Logistic Regression

In [12]:
LR = LogisticRegression(random_state=101, max_iter=1000)

In [13]:
parameters_LR = {"C": np.logspace(-4, 4, 20), "class_weight":[None, "balanced"], "solver": ["liblinear", "lbfgs"], }

In [14]:
#BOW
Randomized_search_LR_BOW = RandomizedSearchCV(LR, parameters_LR, verbose=1, scoring="f1", n_jobs=3)
Randomized_search_LR_BOW.fit(bow_X_train, bow_y_train)
print(" Results from Randomized Search " )
print("\n The best estimator across ALL searched params:\n",Randomized_search_LR_BOW.best_estimator_)
print("\n The best score across ALL searched params:\n",Randomized_search_LR_BOW.best_score_)
print("\n The best parameters across ALL searched params:\n",Randomized_search_LR_BOW.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
 Results from Randomized Search 

 The best estimator across ALL searched params:
 LogisticRegression(C=0.23357214690901212, class_weight='balanced',
                   max_iter=1000, random_state=101)

 The best score across ALL searched params:
 0.7094704229217687

 The best parameters across ALL searched params:
 {'solver': 'lbfgs', 'class_weight': 'balanced', 'C': 0.23357214690901212}


In [15]:
#BOW final
LR_final_BOW = Randomized_search_LR_BOW.best_estimator_
LR_final_BOW.fit(bow_X_train, bow_y_train)
bow_y_pred_LR = LR_final_BOW.predict(bow_X_test)
#test score
f1_score(bow_y_test, bow_y_pred_LR)

0.6960521272518206

In [16]:
#STM
Randomized_search_LR_STM = RandomizedSearchCV(LR, parameters_LR, verbose=1, scoring="f1", n_jobs=3)
Randomized_search_LR_STM.fit(STM_X_train, STM_y_train)
print(" Results from Randomized Search " )
print("\n The best estimator across ALL searched params:\n",Randomized_search_LR_STM.best_estimator_)
print("\n The best score across ALL searched params:\n",Randomized_search_LR_STM.best_score_)
print("\n The best parameters across ALL searched params:\n",Randomized_search_LR_STM.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
 Results from Randomized Search 

 The best estimator across ALL searched params:
 LogisticRegression(C=10000.0, class_weight='balanced', max_iter=1000,
                   random_state=101, solver='liblinear')

 The best score across ALL searched params:
 0.6290436527416495

 The best parameters across ALL searched params:
 {'solver': 'liblinear', 'class_weight': 'balanced', 'C': 10000.0}


In [17]:
#STM final
LR_final_STM = Randomized_search_LR_STM.best_estimator_
LR_final_STM.fit(STM_X_train, STM_y_train)
STM_y_pred_LR = LR_final_STM.predict(STM_X_test)
#test score
f1_score(STM_y_test, STM_y_pred_LR)

0.6299011794708319

In [18]:
#STM selected
Randomized_search_LR_STM_selected = RandomizedSearchCV(LR, parameters_LR, verbose=1, scoring="f1", n_jobs=3)
Randomized_search_LR_STM_selected.fit(STM_X_selected_train, STM_y_selected_train)
print(" Results from Randomized Search " )
print("\n The best estimator across ALL searched params:\n",Randomized_search_LR_STM_selected.best_estimator_)
print("\n The best score across ALL searched params:\n",Randomized_search_LR_STM_selected.best_score_)
print("\n The best parameters across ALL searched params:\n",Randomized_search_LR_STM_selected.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
 Results from Randomized Search 

 The best estimator across ALL searched params:
 LogisticRegression(C=1438.44988828766, class_weight='balanced', max_iter=1000,
                   random_state=101)

 The best score across ALL searched params:
 0.6287241996041536

 The best parameters across ALL searched params:
 {'solver': 'lbfgs', 'class_weight': 'balanced', 'C': 1438.44988828766}


In [19]:
#STM selected final
LR_final_STM_selected = Randomized_search_LR_STM_selected.best_estimator_
LR_final_STM_selected.fit(STM_X_selected_train, STM_y_selected_train)
STM_y_selected_pred_LR = LR_final_STM_selected.predict(STM_X_selected_test)
#test score
f1_score(STM_y_selected_test, STM_y_selected_pred_LR)

0.6308724832214765

In [20]:
#BERTopic
Randomized_search_LR_Bert = RandomizedSearchCV(LR, parameters_LR, verbose=1, scoring="f1", n_jobs=3)
Randomized_search_LR_Bert.fit(Bert_X_train, Bert_y_train)
print(" Results from Randomized Search " )
print("\n The best estimator across ALL searched params:\n",Randomized_search_LR_Bert.best_estimator_)
print("\n The best score across ALL searched params:\n",Randomized_search_LR_Bert.best_score_)
print("\n The best parameters across ALL searched params:\n",Randomized_search_LR_Bert.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
 Results from Randomized Search 

 The best estimator across ALL searched params:
 LogisticRegression(C=1438.44988828766, class_weight='balanced', max_iter=1000,
                   random_state=101, solver='liblinear')

 The best score across ALL searched params:
 0.4381950662727892

 The best parameters across ALL searched params:
 {'solver': 'liblinear', 'class_weight': 'balanced', 'C': 1438.44988828766}


In [21]:
#BERT final 0.48 veel features, 0.42 outlier reduction
LR_final_Bert = Randomized_search_LR_Bert.best_estimator_
LR_final_Bert.fit(Bert_X_train, Bert_y_train)
Bert_y_pred_LR = LR_final_Bert.predict(Bert_X_test)
#test score
f1_score(Bert_y_test, Bert_y_pred_LR)

0.4490925868963396

In [22]:
#Bertopic selected
Randomized_search_LR_Bert_selected = RandomizedSearchCV(LR, parameters_LR, verbose=1, scoring="f1", n_jobs=3)
Randomized_search_LR_Bert_selected.fit(Bert_X_selected_train, Bert_y_selected_train)
print(" Results from Randomized Search " )
print("\n The best estimator across ALL searched params:\n",Randomized_search_LR_Bert_selected.best_estimator_)
print("\n The best score across ALL searched params:\n",Randomized_search_LR_Bert_selected.best_score_)
print("\n The best parameters across ALL searched params:\n",Randomized_search_LR_Bert_selected.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
 Results from Randomized Search 

 The best estimator across ALL searched params:
 LogisticRegression(C=206.913808111479, class_weight='balanced', max_iter=1000,
                   random_state=101)

 The best score across ALL searched params:
 0.4343258759503148

 The best parameters across ALL searched params:
 {'solver': 'lbfgs', 'class_weight': 'balanced', 'C': 206.913808111479}


In [23]:
#Bertopic selected final
LR_final_Bert_selected = Randomized_search_LR_Bert_selected.best_estimator_
LR_final_Bert_selected.fit(Bert_X_selected_train, Bert_y_selected_train)
Bert_y_selected_pred_LR = LR_final_Bert_selected.predict(Bert_X_selected_test)
#test score
f1_score(Bert_y_selected_test, Bert_y_selected_pred_LR)

0.4444444444444445

Linear Support vector Machine

In [24]:
SVM = LinearSVC(random_state=101)

In [25]:
parameters_SVM = {"C": np.logspace(-4, 4, 20), "class_weight":[None, "balanced"], "loss": ["hinge", "squared_hinge"]}

In [26]:
#BOW
Randomized_search_SVM_BOW = RandomizedSearchCV(SVM, parameters_SVM, verbose=1, scoring="f1", n_jobs=3)
Randomized_search_SVM_BOW.fit(bow_X_train, bow_y_train)
print(" Results from Randomized Search " )
print("\n The best estimator across ALL searched params:\n",Randomized_search_SVM_BOW.best_estimator_)
print("\n The best score across ALL searched params:\n",Randomized_search_SVM_BOW.best_score_)
print("\n The best parameters across ALL searched params:\n",Randomized_search_SVM_BOW.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
 Results from Randomized Search 

 The best estimator across ALL searched params:
 LinearSVC(C=0.004832930238571752, class_weight='balanced', loss='hinge',
          random_state=101)

 The best score across ALL searched params:
 0.7106302303056621

 The best parameters across ALL searched params:
 {'loss': 'hinge', 'class_weight': 'balanced', 'C': 0.004832930238571752}


c:\Users\victo\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [27]:
#BOW final
SVM_final_BOW = Randomized_search_SVM_BOW.best_estimator_
SVM_final_BOW.fit(bow_X_train, bow_y_train)
bow_y_pred_SVM = SVM_final_BOW.predict(bow_X_test)
#test score
f1_score(bow_y_test, bow_y_pred_SVM)

c:\Users\victo\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.6908315565031983

In [28]:
#STM
Randomized_search_SVM_STM = RandomizedSearchCV(SVM, parameters_SVM, verbose=1, scoring="f1", n_jobs=3)
Randomized_search_SVM_STM.fit(STM_X_train, STM_y_train)
print(" Results from Randomized Search " )
print("\n The best estimator across ALL searched params:\n",Randomized_search_SVM_STM.best_estimator_)
print("\n The best score across ALL searched params:\n",Randomized_search_SVM_STM.best_score_)
print("\n The best parameters across ALL searched params:\n",Randomized_search_SVM_STM.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
 Results from Randomized Search 

 The best estimator across ALL searched params:
 LinearSVC(C=29.763514416313132, random_state=101)

 The best score across ALL searched params:
 0.5708559544631546

 The best parameters across ALL searched params:
 {'loss': 'squared_hinge', 'class_weight': None, 'C': 29.763514416313132}


In [29]:
#STM final
SVM_final_STM = Randomized_search_SVM_STM.best_estimator_
SVM_final_STM.fit(STM_X_train, STM_y_train)
STM_y_pred_SVM = SVM_final_STM.predict(STM_X_test)
#test score
f1_score(STM_y_test, STM_y_pred_SVM)

0.5746898263027295

In [30]:
#STM selected
Randomized_search_SVM_STM_selected = RandomizedSearchCV(SVM, parameters_SVM, verbose=1, scoring="f1", n_jobs=3)
Randomized_search_SVM_STM_selected.fit(STM_X_selected_train, STM_y_selected_train)
print(" Results from Randomized Search " )
print("\n The best estimator across ALL searched params:\n",Randomized_search_SVM_STM_selected.best_estimator_)
print("\n The best score across ALL searched params:\n",Randomized_search_SVM_STM_selected.best_score_)
print("\n The best parameters across ALL searched params:\n",Randomized_search_SVM_STM_selected.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
 Results from Randomized Search 

 The best estimator across ALL searched params:
 LinearSVC(C=11.288378916846883, class_weight='balanced', random_state=101)

 The best score across ALL searched params:
 0.6283308201032805

 The best parameters across ALL searched params:
 {'loss': 'squared_hinge', 'class_weight': 'balanced', 'C': 11.288378916846883}


In [31]:
#STM selected final
SVM_final_STM_selected = Randomized_search_SVM_STM_selected.best_estimator_
SVM_final_STM_selected.fit(STM_X_selected_train, STM_y_selected_train)
STM_y_selected_pred_SVM = SVM_final_STM_selected.predict(STM_X_selected_test)
#test score
f1_score(STM_y_selected_test, STM_y_selected_pred_SVM)

0.6312759833706427

In [32]:
#BERTopic
Randomized_search_SVM_Bert = RandomizedSearchCV(SVM, parameters_SVM, verbose=1, scoring="f1", n_jobs=3)
Randomized_search_SVM_Bert.fit(Bert_X_train, Bert_y_train)
print(" Results from Randomized Search " )
print("\n The best estimator across ALL searched params:\n",Randomized_search_SVM_Bert.best_estimator_)
print("\n The best score across ALL searched params:\n",Randomized_search_SVM_Bert.best_score_)
print("\n The best parameters across ALL searched params:\n",Randomized_search_SVM_Bert.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
 Results from Randomized Search 

 The best estimator across ALL searched params:
 LinearSVC(C=1.623776739188721, class_weight='balanced', random_state=101)

 The best score across ALL searched params:
 0.43093745308429127

 The best parameters across ALL searched params:
 {'loss': 'squared_hinge', 'class_weight': 'balanced', 'C': 1.623776739188721}


In [33]:
#BERT final 0.47, 0.42 reduction
SVM_final_Bert = Randomized_search_SVM_Bert.best_estimator_
SVM_final_Bert.fit(Bert_X_train, Bert_y_train)
Bert_y_pred_SVM = SVM_final_Bert.predict(Bert_X_test)
#test score
f1_score(Bert_y_test, Bert_y_pred_SVM)

0.4414575866188769

In [34]:
#Bertopic selected
Randomized_search_SVM_Bert_selected = RandomizedSearchCV(SVM, parameters_SVM, verbose=1, scoring="f1", n_jobs=3)
Randomized_search_SVM_Bert_selected.fit(Bert_X_selected_train, Bert_y_selected_train)
print(" Results from Randomized Search " )
print("\n The best estimator across ALL searched params:\n",Randomized_search_SVM_Bert_selected.best_estimator_)
print("\n The best score across ALL searched params:\n",Randomized_search_SVM_Bert_selected.best_score_)
print("\n The best parameters across ALL searched params:\n",Randomized_search_SVM_Bert_selected.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
 Results from Randomized Search 

 The best estimator across ALL searched params:
 LinearSVC(C=4.281332398719396, class_weight='balanced', loss='hinge',
          random_state=101)

 The best score across ALL searched params:
 0.4299033324945922

 The best parameters across ALL searched params:
 {'loss': 'hinge', 'class_weight': 'balanced', 'C': 4.281332398719396}


c:\Users\victo\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [35]:
#Bertopic selected final
SVM_final_Bert_selected = Randomized_search_SVM_Bert_selected.best_estimator_
SVM_final_Bert_selected.fit(Bert_X_selected_train, Bert_y_selected_train)
Bert_y_selected_pred_SVM = SVM_final_Bert_selected.predict(Bert_X_selected_test)
#test score
f1_score(Bert_y_selected_test, Bert_y_selected_pred_SVM)

c:\Users\victo\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.4398976982097187

XGBClassifier

In [36]:
XGB = XGBClassifier(verbosity = 1, seed = 101, use_label_encoder=False)

In [37]:

parameters_XGB = {"colsample_bytree:": np.arange(0.5,1,0.1) ,"min_child_weight": np.arange(1,10,1), "eta": np.arange(0.01,0.3,0.05), "gamma": np.arange(0,5,1), "max_depth": np.arange(3,10,1), "subsample": np.arange(0.5,1,0.1), "scale_pos_weight": [1, 4.045069258], "objective": ["binary:logistic", "binary:logitraw", "binary:hinge"]}

In [38]:
#BOW
Randomized_search_XGB = RandomizedSearchCV(XGB, parameters_XGB, verbose=1, scoring="f1", n_jobs=3)
Randomized_search_XGB.fit(bow_X_train, bow_y_train)
print(" Results from Randomized Search " )
print("\n The best estimator across ALL searched params:\n",Randomized_search_XGB.best_estimator_)
print("\n The best score across ALL searched params:\n",Randomized_search_XGB.best_score_)
print("\n The best parameters across ALL searched params:\n",Randomized_search_XGB.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[18:33:52] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "colsample_bytree:" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[18:33:52] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
 Results from Randomized Search 

 The best estimator across ALL searched params:
 XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              col

In [39]:
#BOW final
XGB_final_BOW = Randomized_search_XGB.best_estimator_
XGB_final_BOW.fit(bow_X_train, bow_y_train)
bow_y_pred_XGB = XGB_final_BOW.predict(bow_X_test)
#test score
f1_score(bow_y_test, bow_y_pred_XGB)

[18:33:54] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "colsample_bytree:" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[18:33:54] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.6633048875096975

In [40]:
#STM
Randomized_search_XGB_STM = RandomizedSearchCV(XGB, parameters_XGB, verbose=1, scoring="f1", n_jobs=3)
Randomized_search_XGB_STM.fit(STM_X_train, STM_y_train)
print(" Results from Randomized Search " )
print("\n The best estimator across ALL searched params:\n",Randomized_search_XGB_STM.best_estimator_)
print("\n The best score across ALL searched params:\n",Randomized_search_XGB_STM.best_score_)
print("\n The best parameters across ALL searched params:\n",Randomized_search_XGB_STM.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[18:37:08] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "colsample_bytree:" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[18:37:08] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.4.0, the default evaluation metric used with the objective 'binary:logitraw' was changed from 'auc' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
 Results from Randomized Search 

 The best estimator across ALL searched params:
 XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsa

In [41]:
#STM final
XGB_final_STM = Randomized_search_XGB_STM.best_estimator_
XGB_final_STM.fit(STM_X_train, STM_y_train)
XGB_y_pred_STM = XGB_final_STM.predict(STM_X_test)
#test score
f1_score(STM_y_test, XGB_y_pred_STM)

[18:37:14] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "colsample_bytree:" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[18:37:14] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.4.0, the default evaluation metric used with the objective 'binary:logitraw' was changed from 'auc' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.6406858924395947

In [42]:
#STM selected
Randomized_search_XGB_STM_selected = RandomizedSearchCV(XGB, parameters_XGB, verbose=1, scoring="f1", n_jobs=3)
Randomized_search_XGB_STM_selected.fit(STM_X_selected_train, STM_y_selected_train)
print(" Results from Randomized Search " )
print("\n The best estimator across ALL searched params:\n",Randomized_search_XGB_STM_selected.best_estimator_)
print("\n The best score across ALL searched params:\n",Randomized_search_XGB_STM_selected.best_score_)
print("\n The best parameters across ALL searched params:\n",Randomized_search_XGB_STM_selected.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


c:\Users\victo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[18:40:32] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "colsample_bytree:" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[18:40:32] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.4.0, the default evaluation metric used with the objective 'binary:logitraw' was changed from 'auc' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
 Results from Randomized Search 

 The best estimator across ALL searched params:
 XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1,
              colsample_by

In [43]:
#STM selected final
XGB_final_STM_selected = Randomized_search_XGB_STM_selected.best_estimator_
XGB_final_STM_selected.fit(STM_X_selected_train, STM_y_selected_train)
STM_y_selected_pred_XGB = XGB_final_STM_selected.predict(STM_X_selected_test)
#test score
f1_score(STM_y_selected_test, STM_y_selected_pred_XGB)

[18:40:40] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "colsample_bytree:" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[18:40:41] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.4.0, the default evaluation metric used with the objective 'binary:logitraw' was changed from 'auc' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.6512554802710242

In [44]:
#BERTopic
Randomized_search_XGB_Bert = RandomizedSearchCV(XGB, parameters_XGB, verbose=2, scoring="f1", n_jobs=3)
Randomized_search_XGB_Bert.fit(Bert_X_train, Bert_y_train)
print(" Results from Randomized Search " )
print("\n The best estimator across ALL searched params:\n",Randomized_search_XGB_Bert.best_estimator_)
print("\n The best score across ALL searched params:\n",Randomized_search_XGB_Bert.best_score_)
print("\n The best parameters across ALL searched params:\n",Randomized_search_XGB_Bert.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[18:43:13] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "colsample_bytree:" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[18:43:13] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
 Results from Randomized Search 

 The best estimator across ALL searched params:
 XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              col

In [45]:
#BERT final
XGB_final_Bert = Randomized_search_XGB_Bert.best_estimator_
XGB_final_Bert.fit(Bert_X_train, Bert_y_train)
Bert_y_pred_XGB = XGB_final_Bert.predict(Bert_X_test)
#test score
f1_score(Bert_y_test, Bert_y_pred_XGB)

[18:43:18] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "colsample_bytree:" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[18:43:19] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.48414376321353064

In [46]:
#Bertopic selected
Randomized_search_XGB_Bert_selected = RandomizedSearchCV(XGB, parameters_XGB, verbose=1, scoring="f1", n_jobs=3)
Randomized_search_XGB_Bert_selected.fit(Bert_X_selected_train, Bert_y_selected_train)
print(" Results from Randomized Search " )
print("\n The best estimator across ALL searched params:\n",Randomized_search_XGB_Bert_selected.best_estimator_)
print("\n The best score across ALL searched params:\n",Randomized_search_XGB_Bert_selected.best_score_)
print("\n The best parameters across ALL searched params:\n",Randomized_search_XGB_Bert_selected.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


c:\Users\victo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[18:45:42] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "colsample_bytree:" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[18:45:42] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
 Results from Randomized Search 

 The best estimator across ALL searched params:
 XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, colsample_bytree:=0.6,
 

In [47]:
#Bertopic selected final
XGB_final_Bert_selected = Randomized_search_XGB_Bert_selected.best_estimator_
XGB_final_Bert_selected.fit(Bert_X_selected_train, Bert_y_selected_train)
Bert_y_selected_pred_XGB = XGB_final_Bert_selected.predict(Bert_X_selected_test)
#test score
f1_score(Bert_y_selected_test, Bert_y_selected_pred_XGB)

[18:45:46] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "colsample_bytree:" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[18:45:46] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.4775510204081633